In [1]:
import os
import sys
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
from utils.progress.log_progress import log_progress
from utils.files.file_helper import ensure_directory, get_all_files_from_subfolders, save_binary_file
import utils.configuration
import numpy as np
import pandas as pd

In [3]:
config = utils.configuration.Configuration()

In [4]:
HISTORICAL_DATA_FOLDER = config.config['DEFAULT']['FLIGHT_DATA']
WEATHER_DATA_FOLDER = config.config['DEFAULT']['WEATHER_DATA']
MERGED_DATA_LOCATION = config.config['DEFAULT']['MERGED_DATA_LOCATION']
MERGED_DATA_FILE = config.config['DEFAULT']['MERGED_DATA_FILE']
MERGED_DATA_FILE_BIN = config.config['DEFAULT']['MERGED_DATA_FILE_BIN']

In [5]:
def get_all_historical_data(historical_data_folder):
    logging.info('Loading historical data')
    all_data_files = get_all_files_from_subfolders(historical_data_folder)
    
    all_data_array = []
    
    str_array = ['OP_UNIQUE_CARRIER', 'TAIL_NUM', 'ORIGIN', 'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME', 'CRS_ELAPSED_TIME']
    
    for data_file in log_progress(all_data_files):
        data = pd.read_csv(data_file, 
                           dtype={'QUARTER': int, 'FL_NUM': str,
                                 'OP_UNIQUE_CARRIER': str, 'TAIL_NUM': str,
                                 'ORIGIN_AIRPORT_ID': int, 'ORIGIN_AIRPORT_SEQ_ID': int,
                                 'ORIGIN_AIRPORT_SEQ_ID': int, 'ORIGIN_CITY_MARKET_ID': int,
                                 'ORIGIN': str, 'ORIGIN_CITY_NAME': str, 
                                 'DEST_AIRPORT_ID': int, 'DEST_AIRPORT_SEQ_ID': int,
                                 'DEST_CITY_MARKET_ID': int, 'DEST': str,
                                 'DEST_CITY_NAME': str, 'CRS_DEP_TIME': int,
                                 'DEP_DELAY': float, 'ARR_DELAY': float,
                                 'CANCELLED': int, 'CANCELLATION_CODE': str,
                                 'DIVERTED': int, 'CRS_ELAPSED_TIME': str,
                                 'CARRIER_DELAY': float, 'WEATHER_DELAY': float,
                                 'NAS_DELAY': float, 'SECURITY_DELAY': float,
                                 'LATE_AIRCRAFT_DELAY': float
                                 }, 
                           parse_dates=['FL_DATE'])
        
        
        data[str_array] = data[str_array].astype(str)
        
        all_data_array.append(data)
    
    logging.info('Concating loaded historical data chunks')
    all_data = pd.concat(all_data_array)

    return pd.concat(all_data_array)

In [6]:
def preprocess_historical_data(original_data):
    logging.info('Preprocessing historical data')
    
    original_data = original_data.drop(['ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 
                                        'ORIGIN_CITY_MARKET_ID', 'DEST_AIRPORT_ID',
                                        'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID',
                                        'CANCELLATION_CODE', 'Unnamed: 27'], axis=1)
    original_data.update(original_data[['DEP_DELAY','ARR_DELAY','CARRIER_DELAY', 
                                       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
                                       'LATE_AIRCRAFT_DELAY']].fillna(0))
    return original_data

In [7]:
def get_all_weather_data(weather_data_folder):
    logging.info('Loading weather data')
    all_data_files = get_all_files_from_subfolders(weather_data_folder)
    
    all_data_array = []
        
    for data_file in log_progress(all_data_files):
        data = pd.read_csv(data_file, 
                           dtype={'attributes': str, 'datatype': str, 'station': str,
                                  'value': int}, 
                           parse_dates=['date'])
        
        data = data.pivot_table(index=['date'], columns='datatype', values='value').reset_index()
        
        data['city'] = data_file[data_file.rfind("/")+1:][:-4]
        
        all_data_array.append(data)
    
    logging.info('Concating loaded weather data chunks')
    all_data = pd.concat(all_data_array)

    return pd.concat(all_data_array)

In [8]:
def preprocess_weather_data(weather_data):
    selected_cols = ['city', 'date', 'AWND', 'SNOW', 'SNWD', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11']
    
    return weather_data[selected_cols]

In [9]:
def merge_data(historical_data, weather_data):
    logging.info('Preparing origin data')
    origin_weather = weather_data.copy()
    origin_weather = origin_weather.rename(columns={'date': 'FL_DATE', 'city': 'ORIGIN_CITY_NAME'})
    origin_cols = origin_weather.columns[~origin_weather.columns.str.contains('ORIGIN_CITY_NAME|FL_DATE')]
    origin_weather.rename(columns = dict(zip(origin_cols, 'origin_' + origin_cols)), inplace=True)
    
    
    logging.info('Preparing dest data')
    dest_weather = weather_data.copy()
    dest_weather = dest_weather.rename(columns={'date': 'FL_DATE', 'city': 'DEST_CITY_NAME'})
    dest_weather[dest_weather.columns.difference(['DEST_CITY_NAME', 'FL_DATE'])] = dest_weather[dest_weather.columns.difference(['DEST_CITY_NAME', 'FL_DATE'])].add_prefix('dest_')
    dest_cols = dest_weather.columns[~dest_weather.columns.str.contains('DEST_CITY_NAME|FL_DATE')]
    dest_weather.rename(columns = dict(zip(dest_cols, 'dest_' + dest_cols)), inplace=True)
   
    
    
    merged_data = historical_data.copy()
    
    merged_data.ORIGIN_CITY_NAME = merged_data.ORIGIN_CITY_NAME.astype(str)
    merged_data.DEST_CITY_NAME = merged_data.DEST_CITY_NAME.astype(str)
    origin_weather.ORIGIN_CITY_NAME = origin_weather.ORIGIN_CITY_NAME.astype(str)
    dest_weather.DEST_CITY_NAME = dest_weather.DEST_CITY_NAME.astype(str)
    
    logging.info('Merging with origin data')
    merged_data = pd.merge(merged_data, origin_weather, on=['ORIGIN_CITY_NAME', 'FL_DATE'], how='left')
    
    logging.info('Merging with dest data')
    merged_data = pd.merge(merged_data, dest_weather, on=['DEST_CITY_NAME', 'FL_DATE'], how='left')
    
    return merged_data
   

In [10]:
def perform_merge_process(historical_data_folder, weather_data_folder, merged_data_location, merged_data_csv_name, merged_data_bin_name):
    logging.info('Starting to merge data')
    historical_data = get_all_historical_data(historical_data_folder)
    historical_data = preprocess_historical_data(historical_data)
    
    weather_data = get_all_weather_data(weather_data_folder)
    weather_data = preprocess_weather_data(weather_data)
    
    merged_data = merge_data(historical_data, weather_data)
    
    logging.info('Total number of docs {}'.format(len(historical_data)))
    logging.info('Merged number of docs {}'.format(len(merged_data)))
    
    logging.info('Saving merged data to csv')
    merged_data.to_csv('{}/{}'.format(merged_data_location, merged_data_csv_name) 
                       ,chunksize=100000
                       ,compression='gzip'
                       ,index=False)
    
    logging.info('Saving merged data to binary file')
    save_binary_file(merged_data, merged_data_bin_name, merged_data_location)
    
    return merged_data

In [11]:
zz = perform_merge_process(HISTORICAL_DATA_FOLDER, WEATHER_DATA_FOLDER, MERGED_DATA_LOCATION, MERGED_DATA_FILE, MERGED_DATA_FILE_BIN)

INFO:root:Starting to merge data
INFO:root:Loading historical data


INFO:root:Concating loaded historical data chunks
INFO:root:Preprocessing historical data
INFO:root:Loading weather data


INFO:root:Concating loaded weather data chunks
INFO:root:Preparing origin data
INFO:root:Preparing dest data
INFO:root:Merging with origin data
INFO:root:Merging with dest data
INFO:root:Total number of docs 13531490
INFO:root:Merged number of docs 13548434
INFO:root:Saving merged data to csv
INFO:root:Saving merged data to binary file
